In [6]:
import os
import numpy as np
import pandas as pd
import cvxpy as cp

In [10]:
import pandas as pd

# 读取CSV文件
gupiao = pd.read_csv('股票_portfolio_return.csv', encoding='utf-8-sig', index_col='date')

date_df = gupiao.index
date_df
print(date_df.shape)

(223,)


In [11]:
import dask.dataframe as dd
import pandas as pd
from dask import delayed, compute
from joblib import Parallel, delayed
import numpy as np
import cvxpy as cp
from rtree import index
from tqdm import tqdm
from geopy.distance import geodesic
import os
import csv

# 初始变量
result_files_pre = []
result_files = []
portfolio_returns_dict = {}  # 存储每个 i_ori 的 portfolio_return_value

types = ['股票', '基金', '期货', '国债']
os_path = "E:\\大三上学科\\金融大数据分析与量化交易\\作业\\hw2\\international"

for i_ori in types:
    def convert_dates(date_series):
        def convert_single_date(date_str):
            try:
                # 尝试转换中文日期格式
                return pd.to_datetime(date_str, format='%Y年%m月%d日')
            except ValueError:
                # 如果转换失败，假设日期已经是标准格式
                return pd.to_datetime(date_str)
        
        # 对日期序列中的每个日期进行转换
        return date_series.apply(convert_single_date)

    data = pd.DataFrame()

    i = os.path.join(os_path, i_ori)

    kinds = os.listdir(i)

    print(kinds)

    for j in range(len(kinds)):
        entire_path = os.path.join(i, kinds[j])
        if j == 0:
            data = pd.read_csv(entire_path, usecols=['日期', '收盘'])
            data = data.rename(columns={'日期': 'date', '收盘': kinds[j].split('.csv')[0]})
            data['date'] = convert_dates(data['date'])
        else:
            temp_df = pd.read_csv(entire_path, usecols=['日期', '收盘'])
            temp_df = temp_df.rename(columns={'日期': 'date', '收盘': kinds[j].split('.csv')[0]})
            temp_df['date'] = convert_dates(temp_df['date'])
            data = pd.merge(data, temp_df, on='date')

        print(data.head())

    data.set_index('date', inplace=True)
    # 将日期从倒序转为正序
    data = data.sort_index(ascending=True)

    data = data[data.index.isin(date_df)]

    if i_ori != '国债':
        data = data.replace(',', '', regex=True).apply(pd.to_numeric)
        returns = data.pct_change().dropna()

        # # 对 returns 中的每个值进行 (returns.shape[0]) 次方运算
        # n = returns.shape[0]
        # returns = (1+returns) ** n - 1


        print(returns.head(-5))
    else:
        returns = data.copy()
        returns = returns.iloc[1:]

    returns.to_csv(os.path.join(os_path, i_ori + '_returns.csv'))
    print(f"{i_ori} 收益率summary:")

    mean_returns = returns.mean()
    print("平均收益率:")
    print(mean_returns)

    std_devs = returns.std()
    print("标准差:\n")
    print(std_devs)

    cov_matrix = returns.cov()
    print("协方差矩阵:")
    print(cov_matrix)

    corr_matrix = returns.corr()
    print("相关系数矩阵:")
    print(corr_matrix)

    alpha = 0.99
    n = returns.shape[1]
    w = cp.Variable(n)
    a = cp.Variable()
    exceedance = cp.Variable(returns.shape[0], nonneg=True)
    portfolio_return = returns.values @ w
    constraints = [cp.sum(w) == 1, w >= 0]
    constraints += [cp.sum(exceedance) <= (1 - alpha) * len(returns)]
    constraints += [exceedance >= portfolio_return - a]
    positive_diff = cp.maximum(-portfolio_return + a, 0)
    expected_positive_diff = cp.mean(positive_diff)
    objective = cp.Minimize(a + (1 / (1 - alpha)) * expected_positive_diff)
    problem = cp.Problem(objective, constraints)
    problem.solve()

    expected_portfolio_return = mean_returns.values @ w.value 
    expected_portfolio_risk = np.sqrt(w.value.T @ cov_matrix.values @ w.value)

    print(f"{i_ori} Optimal Weights:", w.value)
    portfolio_return_value = returns.values @ w.value
    portfolio_returns_dict[i_ori] = portfolio_return_value

    with open(os.path.join(os_path, i_ori) + '_final_results.txt', 'w') as f:
        f.write("平均收益率:\n")
        f.write(mean_returns.to_string())
        f.write("\n\n标准差:\n")
        f.write(std_devs.to_string())
        f.write("\n\n协方差矩阵:\n")
        f.write(cov_matrix.to_string())
        f.write("\n\n相关系数矩阵:\n")
        f.write(corr_matrix.to_string())
        f.write("\n\n------------------------------\n")
        f.write("\nOptimal Weights:\n") 
        f.write(str(w.value)) 
        f.write("\n\nExpected Portfolio Return:\n") 
        f.write(str(expected_portfolio_return)) 
        f.write("\n\nExpected Portfolio Risk (Standard Deviation):\n") 
        f.write(str(expected_portfolio_risk))

    print(f"{i_ori} 结果已写入 results.txt 文件")

    # 创建新的DataFrame用于存储每个i_ori的portfolio_return_value并将其保存到CSV文件
    dfdd = pd.DataFrame({i_ori: portfolio_returns_dict[i_ori]})
    if i_ori == '国债':
        dfdd['date'] = data.index[1:]
    else:
        dfdd['date'] = data.index[1:]
    dfdd.set_index('date', inplace=True)
    dfdd.to_csv(os.path.join(os_path, i_ori + '_portfolio_return.csv'))

# 将所有portfolio_return_value汇总到一个新的DataFrame中 
portfolio_returns_df = pd.DataFrame(portfolio_returns_dict) 
portfolio_returns_df.index = date_df[1:]
# 保存到CSV文件中 
portfolio_returns_df.to_csv(os.path.join(os_path, 'portfolio_returns_summary.csv'), encoding='utf_8_sig') 
print("所有组合收益率已汇总并保存到 portfolio_returns_summary.csv 文件中")


['MSCI新兴市场指数历史数据.csv', 'SPY历史数据.csv', '欧洲斯托克(Eurostoxx)50指数历史数据.csv']
        date MSCI新兴市场指数历史数据
0 2024-12-06       1,104.23
1 2024-12-05       1,102.29
2 2024-12-04       1,100.49
3 2024-12-03       1,097.52
4 2024-12-02       1,086.39
        date MSCI新兴市场指数历史数据  SPY历史数据
0 2024-12-06       1,104.23   607.81
1 2024-12-05       1,102.29   606.66
2 2024-12-04       1,100.49   607.66
3 2024-12-03       1,097.52   603.91
4 2024-12-02       1,086.39   603.63
        date MSCI新兴市场指数历史数据  SPY历史数据 欧洲斯托克(Eurostoxx)50指数历史数据
0 2024-12-06       1,104.23   607.81                 4,977.35
1 2024-12-05       1,102.29   606.66                 4,951.58
2 2024-12-04       1,100.49   607.66                 4,919.02
3 2024-12-03       1,097.52   603.91                 4,878.51
4 2024-12-02       1,086.39   603.63                 4,846.73
            MSCI新兴市场指数历史数据   SPY历史数据  欧洲斯托克(Eurostoxx)50指数历史数据
date                                                          
2023-11-09       -0.001733 -0.007799      

C:\Users\86183\AppData\Local\Temp\ipykernel_53064\1947673569.py:60: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  data = data[data.index.isin(date_df)]


        date  003095历史数据  IBUY历史数据  SKYY历史数据
0 2024-12-06       1.663     69.58    130.46
1 2024-12-05       1.621     68.38    126.88
2 2024-12-04       1.631     69.62    127.62
3 2024-12-03       1.651     68.62    123.33
4 2024-12-02       1.657     68.59    122.54
            003095历史数据  IBUY历史数据  SKYY历史数据
date                                      
2023-11-09   -0.012327 -0.025328 -0.016413
2023-11-10    0.000499  0.010486  0.020395
2023-11-13   -0.009481 -0.005640 -0.003634
2023-11-14    0.000504  0.050136  0.037905
2023-11-15    0.006042  0.026572  0.001757
...                ...       ...       ...
2024-11-21   -0.004174  0.011453  0.028226
2024-11-22   -0.040719  0.017292  0.016620
2024-11-25   -0.003121  0.029633  0.005775
2024-11-26    0.003131 -0.004821  0.002183
2024-11-27    0.021223 -0.001615 -0.019850

[217 rows x 3 columns]
基金 收益率summary:
平均收益率:
003095历史数据   -0.000671
IBUY历史数据      0.002078
SKYY历史数据      0.002499
dtype: float64
标准差:

003095历史数据    0.021483
IBUY历史数据    

C:\Users\86183\AppData\Local\Temp\ipykernel_53064\1947673569.py:60: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  data = data[data.index.isin(date_df)]


        date  WTI原油期货历史数据  玉米期货历史数据  美国小麦期货历史数据
0 2024-12-06        67.39    440.00      556.12
1 2024-12-05        68.30    435.00      558.25
2 2024-12-04        68.54    430.00      548.25
3 2024-12-03        69.94    432.25      547.50
4 2024-12-02        68.10    432.50      547.25
        date  WTI原油期货历史数据  玉米期货历史数据  美国小麦期货历史数据  黄金期货历史数据
0 2024-12-06        67.39    440.00      556.12  2,656.89
1 2024-12-05        68.30    435.00      558.25  2,648.40
2 2024-12-04        68.54    430.00      548.25  2,676.20
3 2024-12-03        69.94    432.25      547.50  2,667.90
4 2024-12-02        68.10    432.50      547.25  2,658.50
            WTI原油期货历史数据  玉米期货历史数据  美国小麦期货历史数据  黄金期货历史数据
date                                                   
2023-11-09     0.005443 -0.016807   -0.019417  0.005965
2023-11-10     0.018880 -0.008547   -0.009471 -0.016182
2023-11-13     0.014125  0.028556    0.006519  0.006385
2023-11-14     0.000000  0.002095   -0.012090  0.008223
2023-11-15    -0.020445  0.0

C:\Users\86183\AppData\Local\Temp\ipykernel_53064\1947673569.py:60: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  data = data[data.index.isin(date_df)]


        date  中国一年期国债收益率历史数据  保加利亚一年期国债收益率历史数据
0 2024-12-06           1.356              3.15
1 2024-12-05           1.350              3.15
2 2024-12-04           1.351              3.55
3 2024-12-02           1.387              3.15
4 2024-11-29           1.389              3.15
        date  中国一年期国债收益率历史数据  保加利亚一年期国债收益率历史数据  澳大利亚一年期国债收益率历史数据
0 2024-12-06           1.356              3.15             4.099
1 2024-12-05           1.350              3.15             4.082
2 2024-12-04           1.351              3.55             4.072
3 2024-12-02           1.387              3.15             4.149
4 2024-11-29           1.389              3.15             4.139
        date  中国一年期国债收益率历史数据  保加利亚一年期国债收益率历史数据  澳大利亚一年期国债收益率历史数据  \
0 2024-12-06           1.356              3.15             4.099   
1 2024-12-05           1.350              3.15             4.082   
2 2024-12-04           1.351              3.55             4.072   
3 2024-12-02           1.387              3.15          

C:\Users\86183\AppData\Local\Temp\ipykernel_53064\1947673569.py:60: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  data = data[data.index.isin(date_df)]
